In [101]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import sparse
from scipy.sparse import csr_matrix
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight, BM25Recommender
from implicit.als import AlternatingLeastSquares
from IPython.core.interactiveshell import InteractiveShell
import time

InteractiveShell.ast_node_interactivity = 'all'

In [92]:
class DataLoader:
    def __init__(self, userData:Path , restaurantData: Path) -> None:
        self.userData = userData
        self.restaurantData = restaurantData
        self.restaurantDF = None

    def load_user_data(self) -> csr_matrix:
        df = pd.read_csv(self.userData, sep=",")

        df.set_index(['userID', 'restaurantID'], inplace=True)
        coo = sparse.coo_matrix(
            (
                df["rating"].astype(float),
                (df.index.get_level_values(0), df.index.get_level_values(1)),
            )
        )

        return coo.tocsr()

    def get_restaurant(self, rest_id: int) -> str:
        return self.restaurantData.iloc[rest_id]['name']

    def load_restaurant_data(self) -> None:
        df = pd.read_csv(self.restaurantData, sep=',')
        df = df.set_index('id')
        
        df['score'] = df['score'].fillna(np.random.uniform(0, 10))
        df['ratings'] = df['ratings'].fillna(np.random.randint(0, 100))
        df['price_range'] = df['price_range'].fillna(np.random.randint(1,2) * "$")

        df['zip_code'] = df['zip_code'].str.split('-', n=1, expand=True)[0]
        # df['zip_code']
        self.restaurantData = df
        return 



In [102]:
class Model:
    def __init__(self):
      self.model= AlternatingLeastSquares(factors=128, iterations=10, regularization=0.01)
      
    
    def calculate_recommendation(self, user_items: csr_matrix):
      artists, _ , plays = get_lastfm() 

      start = time.time()
      self.model.fit(user_items)
      print(f"Finished training the model at {time.time() - start}")


In [103]:
loader = DataLoader(Path('user-data.csv'), Path('restaurants.csv'))
csr = loader.load_user_data()
loader.load_restaurant_data()

model = Model()
model.calculate_recommendation(csr)


100%|██████████| 10/10 [00:18<00:00,  1.83s/it]

Finished training the model at 18.31993293762207
